In [5]:
import os, torch, csv
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import AutoImageProcessor, AutoModelForImageClassification
from sklearn.utils.class_weight import compute_class_weight

# ========== PATHS ==========
base_path = "/Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project"
dataset_path = os.path.join(base_path, "OCTID", "Train")
logs_base = os.path.join(base_path, "logs", "swin")
model_save_path = os.path.join(logs_base, "models", "swin_octid.pth")
log_file = os.path.join(logs_base, "training", "swin_training_log.csv")

# Create output folders
os.makedirs(os.path.join(logs_base, "training"), exist_ok=True)
os.makedirs(os.path.join(logs_base, "models"), exist_ok=True)

# ========== DEVICE & MODEL ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = "microsoft/swin-tiny-patch4-window7-224"
processor = AutoImageProcessor.from_pretrained(checkpoint)
model = AutoModelForImageClassification.from_pretrained(checkpoint, num_labels=5, ignore_mismatched_sizes=True)
model.to(device)

# ========== DATASET ==========
class OCTDataset(Dataset):
    def __init__(self, root_dir, processor):
        self.processor = processor
        self.image_paths, self.labels = [], []
        self.class_names = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        for idx, class_name in enumerate(self.class_names):
            class_path = os.path.join(root_dir, class_name)
            for img_file in os.listdir(class_path):
                if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    self.image_paths.append(os.path.join(class_path, img_file))
                    self.labels.append(idx)
    def __len__(self): return len(self.image_paths)
    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        inputs = self.processor(images=img, return_tensors="pt")
        return {
            "pixel_values": inputs["pixel_values"].squeeze(0),
            "labels": torch.tensor(self.labels[idx])
        }

# ========== DATALOADER & LOSS ==========
dataset = OCTDataset(dataset_path, processor)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

class_weights = compute_class_weight('balanced', classes=np.unique(dataset.labels), y=dataset.labels)
weights = torch.tensor(class_weights, dtype=torch.float).to(device)

loss_fn = torch.nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

# ========== TRAINING LOOP ==========
epochs = 15

with open(log_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Epoch", "Train_Loss", "Train_Accuracy"])

for epoch in range(1, epochs + 1):
    model.train()
    total_loss, correct = 0, 0
    total_samples = 0

    for batch in train_loader:
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(pixel_values=pixel_values)
        logits = outputs.logits
        loss = loss_fn(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * pixel_values.size(0)
        correct += (logits.argmax(-1) == labels).sum().item()
        total_samples += pixel_values.size(0)

    avg_loss = total_loss / total_samples
    accuracy = correct / total_samples
    print(f"Epoch {epoch} - Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

    with open(log_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([epoch, avg_loss, accuracy])

# ========== SAVE FINAL MODEL ==========
torch.save(model.state_dict(), model_save_path)
print(f"\n✅ Final model saved to: {model_save_path}")


/Users/mananmathur/anaconda3/envs/swin_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-tiny-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Si

Epoch 1 - Loss: 1.1344, Accuracy: 0.6341
Epoch 2 - Loss: 0.3699, Accuracy: 0.9199
Epoch 3 - Loss: 0.1599, Accuracy: 0.9512
Epoch 4 - Loss: 0.0918, Accuracy: 0.9791
Epoch 5 - Loss: 0.0367, Accuracy: 0.9965
Epoch 6 - Loss: 0.0218, Accuracy: 0.9965
Epoch 7 - Loss: 0.0065, Accuracy: 1.0000
Epoch 8 - Loss: 0.0070, Accuracy: 1.0000
Epoch 9 - Loss: 0.1976, Accuracy: 0.9582
Epoch 10 - Loss: 0.1425, Accuracy: 0.9652
Epoch 11 - Loss: 0.0108, Accuracy: 1.0000
Epoch 12 - Loss: 0.0050, Accuracy: 1.0000
Epoch 13 - Loss: 0.0032, Accuracy: 1.0000
Epoch 14 - Loss: 0.0027, Accuracy: 1.0000
Epoch 15 - Loss: 0.0012, Accuracy: 1.0000

✅ Final model saved to: /Users/mananmathur/Documents/Academics/MIT/subject matter/YEAR 4/SEM 8/PROJECT/project/logs/swin/models/swin_octid.pth


In [6]:
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def evaluate(split_name, split_path, summary_list):
    print(f"\n🔍 Evaluating: {split_name.upper()}")

    dataset = OCTDataset(split_path, processor)
    loader = DataLoader(dataset, batch_size=16, shuffle=False)

    all_preds, all_labels = [], []
    model.eval()
    with torch.no_grad():
        for batch in loader:
            inputs = batch['pixel_values'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(pixel_values=inputs)
            preds = outputs.logits.argmax(dim=-1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    class_names = dataset.class_names
    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
    df = pd.DataFrame(report).transpose()
    df.to_csv(os.path.join(logs_base, "results", f"{split_name.lower()}_classification_report.csv"))

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.title(f"{split_name} Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.savefig(os.path.join(logs_base, "results", f"{split_name.lower()}_confusion_matrix.png"))
    plt.close()

    summary_list.append({
        "Split": split_name,
        "Accuracy": report["accuracy"],
        "Precision": report["weighted avg"]["precision"],
        "Recall": report["weighted avg"]["recall"],
        "F1-score": report["weighted avg"]["f1-score"],
        "Support": int(report["weighted avg"]["support"])
    })

# Run evaluation on all 3 splits
summary = []
evaluate("Train", os.path.join(base_path, "OCTID", "Train"), summary)
evaluate("Validation", os.path.join(base_path, "OCTID", "Validation"), summary)
evaluate("Test", os.path.join(base_path, "OCTID", "Test"), summary)

df_summary = pd.DataFrame(summary)
df_summary.to_excel(os.path.join(logs_base, "results", "swin_summary.xlsx"), index=False)

print("\n✅ Evaluation complete. Results saved to /logs/swin/results/")



🔍 Evaluating: TRAIN

🔍 Evaluating: VALIDATION

🔍 Evaluating: TEST

✅ Evaluation complete. Results saved to /logs/swin/results/
